In [1]:
import json

from tqdm import tqdm
import tweepy
from dotenv import load_dotenv
import os

from tweepy import TweepyException

load_dotenv()  # load environment variables from .env

True

In [2]:
DATA_PATH = 'data'
YEARS = [2016, 2020]

In [16]:
for year in YEARS:
    with open(os.path.join(DATA_PATH, f'polls-{year}.json')) as f:
        polls = json.load(f)
    author_ids = [poll['meta']['author_id'] for poll in filter(lambda x: x['type']=='twitter', polls)]
    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json'), 'w+') as f:
        json.dump(author_ids, f)

In [17]:
auth = tweepy.OAuth2BearerHandler(
    bearer_token= os.environ['BEARER'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [18]:
exception_users = dict()

In [ ]:

for year in YEARS:
    out_path = os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl')
    existing_users = set()
    if os.path.exists(out_path):
        with open(out_path) as f:
            existing_users.update(k for l in f for k in json.loads(l))
            print(f'found {len(existing_users)} scraped users')

    with open(os.path.join(DATA_PATH, f'polls_{year}_author_ids.json')) as f:
        author_ids = [id for id in json.load(f) if id not in existing_users]

    with open(os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl'), 'a+', encoding='utf8') as f:
        for author_id in tqdm(author_ids, desc='checking followers of poll authors'):
            follower_ids = []
            try:
                for page in tweepy.Cursor(api.get_follower_ids, user_id = author_id, count = 5000).pages():
                    follower_ids.extend(page)
                f.write(json.dumps({author_id:follower_ids}) + '\n')
            except TweepyException as e:
                print(e)
                print(author_id)
                exception_users[id]=e


found 95 scraped users


checking followers of poll authors:   0%|          | 1/292 [00:00<01:04,  4.50it/s]

404 Not Found
34 - Sorry, that page does not exist.
2515437133


checking followers of poll authors:   2%|▏         | 5/292 [00:02<02:33,  1.87it/s]Rate limit reached. Sleeping for: 895


In [ ]:
# client = tweepy.Client(bearer_token=os.environ['BEARER'],
#                        wait_on_rate_limit=True)
# author_id = author_ids[0]
# user_fields = ['id']
# response = client.get_users_followers(id=author_id, user_fields=user_fields, max_results=1000)
# one_user = response.data[0]
# one_user.data
# next_token = response.meta['next_token']